# Fixing the filenames
Goal: don't actually move or rename the file but create a json file with structural information 

1. Preparing to fix some file names, by checking how many specific punctuation characters they have

In [1]:
# check where the files are first
#! cd ../../../data/NAIL_DATAFIELD_txt/parsed_v3 && pwd && ls

In [2]:
import os
import json
import codecs
import re
from tqdm import tqdm
from random import choice
from copy import copy

from IPython.display import clear_output

Create a json object valid_files with the category folders as keys (e.g. law_legal_theory_prison_ip) and another dict as values:
```
{
    path: <folder_path>,  
    valid_files: [
        {
            file_name: <file_name>,
            automatic_name: True|False,
            auto_group_assigned: <some_auto_name_group_name> 
            assigned_to: "M"|"R",
            manually_set: True|False,
            author_name: <first_name(,last_name)?>,
        },
        ...
    ]
}
```


A maybe getting better and better auto naming algo is applied. For non auto detected filenames a person is assigned to set the name manually.
For each auto naming of a file the descicion tree is run through:

### Descicion tree:

- auto_name_found:
    - yes: auto_named already?
        - yes: same name?
            - yes: all cool
            - no : warning: dont set new name, only when flag set (override_old_auto)
        - no : manually_named already?
            - yes: same name?
                - yes: all cool
                - no : critical warning. don't set
            - no : cool. set name
    - no : auto_named already?
        - yes: critical warning. change of algo messed auto nameing up
        - no : assign to a person if not assigned already
        

In [3]:
auto_name_groups = ['1comma_1dash','1comma_>1dash','1dash']

two_words = re.compile("([a-zA-Z]+[,-_]){2}")

class FileDescriptor:
    
    def __init__(self, file_name, dict_ = None):
        self.file_name = file_name
        self.auto_named = False # flag indicating that auto name applied
        self.auto_group_name = None # auto name rule name for debugging... 
        self.assigned_to = None # assigned to person (M|R for splitting :D )
        self.manually_named = False # flag indicating that name was set manually
        self.author_name = "" # final author, JUST ONE
        
        if dict_:
            self.from_dict(dict_)
        
    def to_dict(self):
        return self.__dict__.copy()
    
    def set_from_dict(self, dict_):
        self.__dict__ = dict_.copy()
    
    def set_auto_name(self, author_name, auto_group_name, override_old_auto = False, override_manual = False, debug = False):
        """
        contains the desciocion tree. read above
        """
        if debug: print('setting name to',author_name)
        # auto_named_already?
        if self.auto_named:
            if debug: print('auto_named already')
            # yes: same name?
            if self.author_name == author_name:
                # yes: all cool. set the new group anyway
                self.auto_group_name = auto_group_name
            # no : warning: dont set new name, only when flag set
            else:
                # check flag set (override_old_auto)
                if override_old_auto:
                    self.author_name = author_name
                    self.auto_group_name = auto_group_name
                else:
                    print('Warning: New Auto name does not match old one.')
                    print(self.file_name,'old',self.author_name,'new auto',author_name)
                    print('Not gonna take it. check your algo')
        # no (not auto_named_already)
        else: 
            if debug: print('not auto_named yet')
            # manually named already?
            if self.manually_named:
                # yes: same name?
                if self.author_name == author_name:
                    # yes: all cool
                    self.auto_group_name = auto_group_name
                # no : only set when flag set (override_manual). Otherwise warning
                else:
                    if override_manual:
                        self.author_name = author_name
                        self.auto_group_name = auto_group_name
                        self.auto_named = True
                        self.manually_named = False
                    else:
                        print("Warning. Name has been set manually already.")
                        print(self.file_name)
                        print('old',self.author_name,'new auto',author_name)
                        print('Not gonna take it. check your algo')
            else:
                # new find: name that shit!
                self.author_name = author_name
                self.auto_group_name = auto_group_name  
                self.auto_named = True
                        
    def __repr__(self):
        return json.dumps(self.to_dict())
    
    def auto_name_check(self, debug = False):
        """
        THIS IS THE SMART FUNCTION. IF YOU KNOW SOME GOOD RULES TO FIND THE NAME FROM A TXT FILE PUT IT HERE
        """
        f = self.file_name
        auto_name = None
        if debug:
            print('checking',self.file_name)
            print(', :',f.count(','),'   - :',f.count('-'))
        if f.count(',') == 1:
            if debug: print('found  1 comma')
            # FINAL this is a great set. lastname, firstname - title something like ALBERRO, NORVELL-recording_conceptual_art
            if f.count('-') == 1:
                auto_name = (f[:f.find('-')].strip(), '1comma_1dash')
            # FINAL? here we have the words in the title separated with - or a minus in a a word 
            elif f.count('-') > 1:
                auto_name = (f[:f.find('-')].strip(), '1comma_>1dash')
            else:
                pass
        elif f.count(',') > 1:
            pass
        # FINAL. basically lastname - something to separate words in the title
        elif f.count('-') == 1:
            if debug: print('found 1dash')
            auto_name = (f[:f.find('-')].strip(), '1dash')
        elif f.count('-') > 1:
            if debug: print('found  >1 -')
            pass
        # not so many anymore. do them manually. often no author
        else :
            pass
        return auto_name    
  
    def auto_name(self, override_old_auto = False, override_manual = False, debug= False):
        auto_name = self.auto_name_check(debug)
        if auto_name:
            self.set_auto_name(*auto_name, override_old_auto, override_manual, debug)
        # TODO add a warning, when we had a auto_name befor and don't get it anymore

    def manual_name_options(self):
        file_name_alt = self.file_name.replace(' ','')
        options = {'no':'n: no author'}
        find_name_match = two_words.match(file_name_alt)
        if find_name_match:
            potential_name =file_name_alt[:find_name_match.span()[1]-1]
            potential_name = potential_name.replace('-',',')
            potential_name = potential_name.replace('_',',')
            splitguess = 'g: ' + potential_name
            options['guess'] = splitguess
        return options

    def manual_naming(self):
        text = ['set first_name(, last_name)? for',self.file_name,'']
        options = self.manual_name_options()
        text.extend(list(options.values()))
        text = '\n'.join(text)
        name = input(text + '\n\n')
        if name == '':
            return
        else:
            if name == 'n':
                self.author_name = options['no']
            elif name == 'g':
#                 print('selected option:',options[3])
                self.author_name = options['guess'][3:]
            else:
                self.author_name = name
            self.auto_named = False
            # keep auto_name_group so we see, if there was something before
            self.manually_named = True

    def simple_manual_merge(self, other_desrc):
        new_file_descr = copy(self)   
        if other_desrc.manually_named and not new_file_descr.manually_named:
            new_file_descr.manually_named = True
            new_file_descr.author_name = other_desrc.author_name
        return new_file_descr
    
class DescriptorCollection:
    """
    File descriptors for a folder of files
    """
    def __init__(self,folder_name,folder_path):
        self.folder_name = folder_name
        self.folder_path = folder_path
        self.file_descriptors = {}

    def to_dict(self):
        dict_ = self.__dict__.copy()
        dict_['file_descriptors'] = {file_descr: self.file_descriptors[file_descr].to_dict() for file_descr in self.file_descriptors}
        return dict_
            
    def from_dict(self, dict_):
        self.__dict__ = dict_
        self.log_files = [FileDescriptor('',dict_) for dict_ in self.file_descriptors]

    def __repr__(self):
        return json.dumps(self.to_dict())
    
    def short_description(self):
        dict_ = self.to_dict().copy()
        dict_['file_descriptors'] = len(dict_['file_descriptors'])
        return json.dumps(dict_)
    
    def build(self, file_filter = None, file_name_processor= None):
        """
        for initialisation:
        builds a log dict for a list of file in a folder. 
        the actual selection of files can be filtered with a passed function
        for each selected file the empty description is created
        """
        for file_ in os.listdir(self.folder_path):
            if not file_filter or file_filter(file_):
                file_key = file_ 
                if file_name_processor:
                    file_key = file_name_processor(file_)
                self.file_descriptors[file_key] = FileDescriptor(file_)
#         return log_files
    
    def get_descriptor(self, file_name_or_index):
        if type(file_name_or_index) == str:
            return self.file_descriptors[file_name_or_index]
        elif type(file_name_or_index) == int:
            return list(self.file_descriptors.values())[file_name_or_index]
        
    def auto_name_all(self, override_old_auto = False, override_manual = False, debug= False):
        for file_descr in list(self.file_descriptors.values()):
            file_descr.auto_name(override_old_auto, override_manual, debug)
            
    def info(self):
        print('Collection:',self.folder_name)
        print(len(self.file_descriptors), 'docs')
        print(sum([1 for text_descr in list(self.file_descriptors.values()) if text_descr.auto_named]), 'auto named')
        sum_man_named = sum([1 for text_descr in list(self.file_descriptors.values()) if text_descr.manually_named])
        sum_assigned = sum([1 for text_descr in list(self.file_descriptors.values()) if text_descr.assigned_to])
        print(sum_man_named,'/',sum_assigned, 'manually named/assigned')
        
        
    def file_descr_list(self):
        return list(self.file_descriptors.values())
    
    def get_all_missing(self):    
        return [text_descr for text_descr in file_descr_list if not text_descr.author_name]
        
    def get_all_assigned_to(self, name = None, only_missing = False):
        return [text_descr for text_descr in list(self.file_descriptors.values()) 
                if text_descr.assigned_to == name  and not(only_missing and text_descr.manually_named)]
 
    def merge(self, other_collection):
        """
        merge in another collection. should be the same basis, just with different file_descriptior values:
        manually assigned author names... basically
        creates a NEW COLLECTION
        Preference goes to auto_naming, if it doens't exist in the 2nd collection. that means the other has a smarter algo
        
        """
        # TODO
#         for file_descr_name in self.file_descriptors:
#             # file descriptor not in other collection? weird... ignore
#             if file_descr_name not in other_collection:
#                 print(file_descr, 'is not in the 2nd collection. not gonna take it')
#             file_descr1 = self.file_descriptors[file_descr_name]
#             file_descr2 = other_collection.file_descriptors[file_descr_name]
#             new_file_descr = FileDescriptor(file_descr1.file_name)
#             # we need to cover 4 cases. auto-auto, auto-man, man-auto, man-man, 
#             # if file_descr1.auto_named and not file_descr2.auto_named
        
            
    def simple_manual_merge(self, other_collection):
        """
        just slam the file_descr together to complete the authors.
        assumption is that auto_naming is the same and just different assigned descriptors are merges
        Returns a new log
        """
        new_col_desrc = DescriptorCollection(self.folder_name, self.folder_path)
        for file_descr_name in self.file_descriptors:
            file_descr1 = self.file_descriptors[file_descr_name]
            file_descr2 = other_collection.file_descriptors[file_descr_name]
            new_file_descr = file_descr1.simple_manual_merge(file_descr2)
            new_col_desrc.file_descriptors[file_descr_name] = new_file_descr 
        return new_col_desrc
    
    def get_all_auto_named(self):
        return [file_descr for file_descr in file_descr_list if file_descr.auto_named]
    
    def get_all_manualy_named(self):
        return [file_descr for file_descr in file_descr_list if file_descr.manually_named and file_descr.author_name]

Next part is simple.
Initiate the description files for our corpus.

This is only for initialisation. If it's done you can just **load a log file**...
At the end we get a dict, where the keys are foldernames and the values are DescriptorCollections
```
{
    folder_name: {
        <DescriptorCollections>: as_json:
        folder_name: <folder_name>
        folder_path: <folder_path>,
        log_files: <list of FileDescriptors>
    }
},
...
```

In [4]:
def build_descr_folder(base_path, folder_names, file_filter, file_name_processor):
    """
    build multiple folder in one base folder to a dict
    {key: folder_name ; value: {path: folder_path, list of fileDescriptions}}
    """
    descr_collections = []
    for folder_name in folder_names:
        descr_folder = DescriptorCollection(folder_name, base_path + folder_name)
        descr_folder.build(file_filter, file_name_processor)
        descr_collections.append(descr_folder)    
    return {collection.folder_name : collection for collection in descr_collections}    

Next we define some methods to initiate a set of collections, read and dump them to drive

In [5]:
def init_log(base_path, file_filter = None, file_name_processor = None):
    """
    initialises
    """
    folder_names = [obj for obj in os.listdir(base_path) if os.path.isdir(base_path + obj)]
    log = build_descr_folder(main_path, folder_names, file_filter, file_name_processor)
  
    # print(log_files)
    total_size = 0
    for folder in log:
        print(folder)
        print(len(log[folder].file_descriptors),'log files')
        total_size += len(log[folder].file_descriptors)
    print('TOTAL SIZE',total_size)
    return log

        
def read_log():
    try:
        with codecs.open(log,encoding='utf-8') as fin:
            log = json.loads(fin.read())
    except FileNotFoundError:
        update_log_file()
    return log

# def update_txt(file_name, file_path, data):
#     file_log = log.get(file_name,{})
#     file_log = {**file_log, **data}
#     log[file_name] = file_log

def update_log_file(log, log_file_name):
    """
    log is a dict of collection. dump it to drive...
    """
    dict_ = {col: log[col].to_dict() for col in log} 
    with codecs.open(log_file_name,'w', encoding='utf-8') as fout:
        fout.write(json.dumps(dict_, indent=2, ensure_ascii=False))

In [6]:
main_path = '../../../data/NAIL_DATAFIELD_txt/parsed_v3/'

def valid_file_filter(file_name):
    return '_valid' in file_name

def valid_file_name_processor(file_name):
    return file_name[:-len('_valid.txt')]

log = init_log(main_path, valid_file_filter, valid_file_name_processor)
update_log_file(log,'log.json')

# now we can grab a file descriptor either by some index or by it's file name:
file_descr = log['arts_arthistory_aesthetics'].get_descriptor(0)
print(json.dumps(file_descr.to_dict(), indent = 2))
file_descr = log['arts_arthistory_aesthetics'].get_descriptor('Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers')
print(json.dumps(file_descr.to_dict(), indent = 2))

digital_and_internet_theory
732 log files
library_and_archive_theory
57 log files
ecology_climate_permaculture_collapse
887 log files
own_mixed_collection
132 log files
law_legal_theory_prison_ip
599 log files
political_theory_anarchist
771 log files
arts_arthistory_aesthetics
1627 log files
TOTAL SIZE 4805
{
  "auto_group_name": null,
  "assigned_to": null,
  "auto_named": false,
  "manually_named": false,
  "author_name": "",
  "file_name": "MUYBRIDGE-horses_and_other_animals_valid.txt"
}
{
  "auto_group_name": null,
  "assigned_to": null,
  "auto_named": false,
  "manually_named": false,
  "author_name": "",
  "file_name": "Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers_valid.txt"
}


Checking a single file if it would find a name:

In [7]:
file_descr.auto_name_check(debug = True)
# that looks good...

checking Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers_valid.txt
, : 1    - : 2
found  1 comma


('Batsford', '1comma_>1dash')

Let's test the auto naming function, which will if it finds a name and the deciscion tree rules are cool set the "file description" 

In [8]:
# test auto_name
file_descr.auto_name(debug= True)
print(json.dumps(file_descr.to_dict(), indent = 2))

checking Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers_valid.txt
, : 1    - : 2
found  1 comma
setting name to Batsford
not auto_named yet
{
  "auto_group_name": "1comma_>1dash",
  "assigned_to": null,
  "auto_named": true,
  "manually_named": false,
  "author_name": "Batsford",
  "file_name": "Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers_valid.txt"
}


manual naming could work like this

In [9]:
file_descr.manual_naming()
print(json.dumps(file_descr.to_dict(), indent = 2))

set first_name(, last_name)? for
Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers_valid.txt

n: no author
g: Batsford,Gwen

g
{
  "auto_group_name": "1comma_>1dash",
  "assigned_to": null,
  "auto_named": false,
  "manually_named": true,
  "author_name": "Batsford,Gwen",
  "file_name": "Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers_valid.txt"
}


now we go through all files in bulk. Instead of `auto_name_check` we use `auto_name`,
which will call `set_auto_name` in case we found something.
The complete check also allowes us to assign a random 'person name' to each text document which has not been auto_named. At the end, we get an overview


In [18]:
def complete_check(dict_of_collections, assign_equally_to = None, 
                   override_old_auto = False, override_manual = False, debug= False):
    """
    when assign_equally_to contains a list, it will randomly choose one for each file that is not
    auto-named
    """
    for collection in list(dict_of_collections.values()):
        collection.auto_name_all()
        for file_descr in list(collection.file_descriptors.values()):
            if not file_descr.auto_named and assign_equally_to:
                file_descr.assigned_to = choice(assign_equally_to)

complete_check(log,['M','R'])
print()
for col in log:
    log[col].info()
    print()



Warning. Name has been set manually already.
Batsford - Gwen.White-Perspective.A.Guide.For.Artists,.Architects.and.Designers_valid.txt
old Batsford,Gwen new auto Batsford
Not gonna take it. check your algo

Collection: digital_and_internet_theory
732 docs
469 auto named
0 / 263 manually named/assigned

Collection: library_and_archive_theory
57 docs
39 auto named
0 / 18 manually named/assigned

Collection: ecology_climate_permaculture_collapse
887 docs
476 auto named
0 / 411 manually named/assigned

Collection: own_mixed_collection
132 docs
128 auto named
3 / 4 manually named/assigned

Collection: law_legal_theory_prison_ip
599 docs
304 auto named
0 / 295 manually named/assigned

Collection: political_theory_anarchist
771 docs
513 auto named
0 / 258 manually named/assigned

Collection: arts_arthistory_aesthetics
1627 docs
944 auto named
1 / 683 manually named/assigned



Let's have a look at all assigned descriptors of own_mixed_collection

In [11]:
[file_descr for file_descr in list(log['own_mixed_collection'].file_descriptors.values()) if file_descr.assigned_to]
# we could also call `get_all_missing` which returns all file_descr. which don't have an author yet
# log['own_mixed_collection'].get_all_missing()

[{"auto_group_name": null, "assigned_to": "R", "auto_named": false, "manually_named": false, "author_name": "", "file_name": "laboria_cuboniks_xenofeminism_a_politics_for_alienation_valid.txt"},
 {"auto_group_name": null, "assigned_to": "R", "auto_named": false, "manually_named": false, "author_name": "", "file_name": "rosa_menkman_the_glitch_momentum_valid.txt"},
 {"auto_group_name": null, "assigned_to": "R", "auto_named": false, "manually_named": false, "author_name": "", "file_name": "nick_bostrom-are_you_living_in_a-computer_simulation_valid.txt"},
 {"auto_group_name": null, "assigned_to": "M", "auto_named": false, "manually_named": false, "author_name": "", "file_name": "Xenofeminism2_valid.txt"}]

nice... now lets get all of them assigned to 'R'. I added a function for that, to have it handy

In [12]:
assigned_to_R = log['own_mixed_collection'].get_all_assigned_to('R')
assigned_to_R

[{"auto_group_name": null, "assigned_to": "R", "auto_named": false, "manually_named": false, "author_name": "", "file_name": "laboria_cuboniks_xenofeminism_a_politics_for_alienation_valid.txt"},
 {"auto_group_name": null, "assigned_to": "R", "auto_named": false, "manually_named": false, "author_name": "", "file_name": "rosa_menkman_the_glitch_momentum_valid.txt"},
 {"auto_group_name": null, "assigned_to": "R", "auto_named": false, "manually_named": false, "author_name": "", "file_name": "nick_bostrom-are_you_living_in_a-computer_simulation_valid.txt"}]

Now let's manually name them and get the info of that collection again.

In [13]:
for file_descr in assigned_to_R:
    file_descr.manual_naming()
    clear_output()
print('cool all done')
log['own_mixed_collection'].info()

cool all done
Collection: own_mixed_collection
132 docs
128 auto named
3 / 4 manually named/assigned


If you tired you can write the log now. for later you can also just grab those, which are not set yet. 
The second parameter 'only_missing' default False does just that...

In [14]:
assigned_to_R = log['own_mixed_collection'].get_all_assigned_to('R', True)
assigned_to_R
# EMPTY SINCE R did all ot his files...

[]

we want to merge 2 logs. Let's not rely on git with that... :)


In [15]:
def simple_manual_merge_logs(log1, log2):
    final_log = {}
    for col_name in log1:
        if col_name in log2:
            final_log[col_name] = log1[col_name].simple_manual_merge(log2[col_name]) 
        else:
            print('collection', col_name,'is missing')
    return final_log
# simple_manual_merge_logs(log,log)

testing the merge... 
a bit hacky...
- initiate a second log `log2`, 
- do auto naming without assignments
- copy the assignments of `own_mixed_collection` from `log` to `log2` 
- manualy name log2 `own_mixed_collection` for M
- now `log` has all R manualy named and `log2` all Ms
- merge them

In [19]:
# still a bit strange...

# initiate a second log
log2 = init_log(main_path, valid_file_filter, valid_file_name_processor)
# do auto naming without assignments
complete_check(log2)

# copy the assignments of own_mixed_collection from log to log2
for file_descr in log['own_mixed_collection'].file_descriptors:
   assigned_to = log['own_mixed_collection'].file_descriptors[file_descr].assigned_to
   log2['own_mixed_collection'].file_descriptors[file_descr].assigned_to = assigned_to

# manualy name log2 own_mixed_collection for M


assigned_to_M = log2['own_mixed_collection'].get_all_assigned_to('M')
for file_descr in assigned_to_M:
    file_descr.manual_naming()
    clear_output()
    
print('cool all done')
log2['own_mixed_collection'].info()

newLog = simple_manual_merge_logs(log,log2)
newLog['own_mixed_collection'].info() 
newLog['own_mixed_collection'].get_all_missing()

cool all done
Collection: own_mixed_collection
132 docs
128 auto named
1 / 4 manually named/assigned


Collection: own_mixed_collection
132 docs
128 auto named
3 / 4 manually named/assigned


Last final part.

initiate or load.

load a second and merge it in.
run through all collections a manualy fill in missing file descriptions

save it